# import 


In [1]:
import unittest

# data

数据文件已经是处理好的单词，每封邮件一行

# 贝叶斯

In [2]:
#先验概率： 
# 好样本： 
1400/(1400+500)

0.7368421052631579

In [3]:
# 坏样本： 
500/(1400+500)

0.2631578947368421

# src

In [12]:
#tdd
import unittest

# 待编写
#from email_classifier import EmailClassifier
class TestEmailClassifier(unittest.TestCase):
    def setUp(self):
        # notin book please PLEASE
        self.tmp_file = '../data/tmp/tmp.txt'

        #self.spam_dir='../data/processed/spam_500/'
        #self.ham_dir='../data/processed/ham_1400/'

        # spam buy buy this book http www
        self.spam_dir = '../data/test_spam/'
        # 1:ham this is ham please refer this book
        # 2:ham please refer this book
        self.ham_dir = '../data/test_ham/'

        self.clf = EmailClassifier(self.spam_dir, self.ham_dir)

    def test_get_files_form_dir(self):
        a_bad = EmailClassifier.get_files_form_dir(self.spam_dir)
        a_good = EmailClassifier.get_files_form_dir(self.ham_dir)
        self.assertEqual(len(a_bad), 1)
        self.assertEqual(len(a_good), 2)

    # 测试文本解析功能:全部小写，取集合
    # expectation : e ;a = actual
    def test_get_words_from_file(self):
        e = set(['notin', 'book', 'please'])
        a = EmailClassifier.get_words_from_file(self.tmp_file)
        self.assertSetEqual(a, e)

    def test_train(self):
        #clf = EmailClassifier(self.spam_dir, self.ham_dir)
        self.clf.train()
        self.assertEqual(self.clf.total_count['spam'], 6)
        self.assertEqual(self.clf.total_count['ham'], 11)
        self.assertEqual(self.clf.training['ham']['please'], 2)
        self.assertEqual(self.clf.training['spam']['buy'], 1)

    def test_score(self):
        
        def _format_dict(d):
            return {k: round(v, 3) for k, v in d.items()}
            
        a = {
            'ham': round(2 / 3 * 1 / 12 * 3 / 12 * 3 / 12, 7),#7。0.003472222222222222
            'spam': round(1 / 3 * 1 / 7 * 2 / 7 * 1 / 7, 7)# 7
        }
        a = {
            'ham':(2 / 3) * (1 / 12) * (3 / 12) * (3 / 12)*(1000**3), #0.003472222222222222
            'spam': (1 / 3) * (1 / 7) * (2 / 7 )*( 1 / 7) *(1000**3)
        }
        #clf = EmailClassifier(self.spam_dir, self.ham_dir) 
        self.clf.train()
        e = self.clf.score(self.tmp_file)
        # 原来的
        # self.assertDictEqual(a, e)
        self.assertDictEqual(_format_dict(a), _format_dict(e))
        
        

    def test_judge(self):
        #self.clf.train()
        t = self.clf.score(self.tmp_file)
        e = {'ham':0.0034722}# 原始测试
        e = {'ham':3472222.2222222215}# 真实调整
        a = self.clf.judge(t)
        self.assertDictEqual(a, e)
        
    def test_classify(self):
        #t = self.clf.score(self.tmp_file)
        #e = {'ham':0.0034722}
        e = {'ham':3472222.2222222215}# 真实调整
        #a = self.clf.judge(t)
        a = self.clf.classify(self.tmp_file)
        self.assertDictEqual(a, e)
        
        
if __name__ == '__main__':
    # 方式一：python -m unittest exame_test
    #unittest.main()

    # 方式二
    # 构造测试集
    #suite = unittest.TestSuite()
    #suite.addTest(TestEmailClassifier('test_get_words_from_file'))
    ## 执行测试
    #runner = unittest.TextTestRunner()
    #runner.run(suite)

    # 方式三
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(
        TestEmailClassifier)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_classify (__main__.TestEmailClassifier) ... ok
test_get_files_form_dir (__main__.TestEmailClassifier) ... ok
test_get_words_from_file (__main__.TestEmailClassifier) ... ok
test_judge (__main__.TestEmailClassifier) ... ok
test_score (__main__.TestEmailClassifier) ... ok
test_train (__main__.TestEmailClassifier) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.012s

OK


In [292]:
2 / 3 * 1 / 12 * 3 / 12 * 3 / 12

0.003472222222222222

In [290]:
round(2 / 3 * 1 / 12 * 3 / 12 * 3 / 12, 9)

0.003472222

## 运行

凡是以test开头的方法都是测试方法

In [11]:
import re
import io
import os
import glob
from collections import defaultdict


class EmailClassifier:
    '''
    spam:垃圾邮件
    ham: 正常邮件
    '''
    def __init__(self, spam_dir, ham_dir):
        self.CAT = ['ham', 'spam']
        self.spam_list = EmailClassifier.get_files_form_dir(spam_dir)
        self.ham_list = EmailClassifier.get_files_form_dir(ham_dir)
        # 先验
        p_ham = len(self.ham_list) / (len(self.ham_list) + len(self.spam_list))
        self.P = {self.CAT[0]: p_ham, self.CAT[1]: 1 - p_ham}

        # 记录每个类别下每个单词的计数
        self.training = {c: defaultdict(float) for c in self.CAT}
        # 记录每个类别单词总数
        self.total_count = {self.CAT[0]: 0, self.CAT[1]: 0}
        # 可当做超参数
        self.super_m = 1000

    @staticmethod
    def get_words_from_file(file):
        with io.open(file, 'r') as f:
            c = re.findall('\w+', f.read().lower())
        return set(c)

    @staticmethod
    def get_files_form_dir(path):
        return glob.glob(os.path.join(path, '*.*'))

    def train(self):
        # 单词统计
        for t in zip(self.CAT, [self.ham_list, self.spam_list]):
            for s in t[1]:
                words = EmailClassifier.get_words_from_file(s)
                self.total_count[t[0]] += len(words)
                for ww in words:
                    self.training[t[0]][ww] += 1

    def score(self, email_file):
        if self.total_count[self.CAT[0]] == 0 or self.total_count[
                self.CAT[1]] == 0:
            self.train()
        result = self.P.copy()
        for ww in EmailClassifier.get_words_from_file(email_file):
            for cc in self.CAT:
                v = self.training[cc][ww]
                # 乘以 1000，避免数值太小
                #p = (v + 1) / (self.total_count[cc] + 1)
                p = self.super_m*(v + 1) / (self.total_count[cc] + 1)
                #p = (1000*v + 1) / (self.total_count[cc] + 1)
                result[cc] *= p
        
        return result
        # 返回7位小数
        return {k: round(v, 7) for k, v in result.items()}

    #@staticmethod
    def judge(self,score_dict):
        '''二分类'''
        keys = self.CAT #list(score_dict.keys())
        if score_dict[keys[0]] >= score_dict[keys[1]]:
            return {keys[0]: score_dict[keys[0]]}
        else:
            return {keys[1]: score_dict[keys[1]]}

    def classify(self, email_file):
        score = self.score(email_file)
        return self.judge(score)

## 验证集

In [13]:
s = '../data/processed/spam_400/'
h = '../data/processed/ham_1120/'
clf = EmailClassifier(s, h)
clf.train()

In [14]:
clf.total_count

{'ham': 165655, 'spam': 71005}

In [15]:
clf.training['spam']['you']

312.0

In [20]:
# 整体*1000 ： （95, 5, 278, 2)
# 分开1000： (93, 7, 280, 0)
predict(clf,v_h,v_s)

{'spam': 1.8387229565810378e-153}
{'spam': 2.029211278197064e-148}
{'ham': 0.0}
{'ham': 6.23380504773489e-64}
{'ham': 0.0}
{'ham': 0.0}
{'ham': 0.0}
accuracy:0.9815789473684211


(95, 5, 278, 2)

In [17]:
def predict(clf, ham_dir, spam_dir):
    ham_list = EmailClassifier.get_files_form_dir(ham_dir)
    spam_list = EmailClassifier.get_files_form_dir(spam_dir)
    tp,fp,tn,fn = 0,0,0,0

    for hh in ham_list:
        t = clf.classify(hh)
        if 'ham' in t:
            #print(t)
            tn += 1
        else:
            print(t)
            fn += 1
            
    for ss in spam_list:
        t = clf.classify(ss)
        if 'spam' in t:
            tp += 1
        else:
            print(t)
            fp += 1
    
    accuracy = (tp + tn) / (tp+tn+fp+fn)
    print('accuracy:{}'.format(accuracy))
    return tp,fp,tn,fn
    

In [19]:
v_s = '../data/validate/spam/'
v_h = '../data/validate/ham/'

In [209]:
predict(clf,h,s)

(0, 400, 1120, 0)

# doc

## unittest

https://docs.python.org/3/library/unittest.html

### 常用测试断言


In [4]:
assert 1==2,'断言失败'

AssertionError: 断言失败

In [5]:
t_test = unittest.TestCase()

In [6]:
# 基本使用方法
t_test.assertEqual(1,2,'断言失败')

AssertionError: 1 != 2 : 断言失败

In [7]:
t_test.assertCountEqual([1,2,3,1],[3,2,1,1])

In [8]:
#search(r"(\w)+", "abc")
t_test.assertRegex('abc', '(\w)+')
t_test.assertRegex('abc', '\d','失败')

AssertionError: Regex didn't match: '\\d' not found in 'abc' : 失败

# end